In [1]:
import warnings
import pandas as pd
import plotly.express as px
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from aicsimageprocessing import diagnostic_sheet, read_ome_zarr, rescale_image, imgtoprojection
from upath import UPath as Path
import warnings
from aicsimageio import transforms, AICSImage
import nbvv
import os

19-Oct-22 22:49:16 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [3]:
# REMOVES:
# edge cells
# m6m7 singles

df = pd.read_parquet("s3://allencell-hipsc-cytodata/hackathon_manifest_17oct2022.parquet")
df.set_index(df['CellId'].astype(int), inplace=True)

df_sub_fit = df[(df['fits_x']==True) &
               (df['fits_y']==True) &
               (df['fits_z']==True)
              ]

df_sub_fit_center = df_sub_fit[df_sub_fit['edge_flag'] == 0]
dataset = df_sub_fit_center.drop(df_sub_fit_center[df_sub_fit_center['cell_stage']=='M6M7_single'].index)

print(f'Number of cells: {len(dataset)}')
print(f'Number of columns: {len(dataset.columns)}')

Number of cells: 176936
Number of columns: 77


In [21]:
# cells with marked structures
#
# soi = structures of interest

soi_df = pd.concat([
    dataset[(dataset['Structure'] == "microtubules")],
    dataset[(dataset['Structure'] == "mitochondria")],
    dataset[(dataset['Structure'] == "actin bundles")],
    dataset[(dataset['Structure'] == "actin filaments")],
    dataset[(dataset['Structure'] == "centrioles")],
    dataset[(dataset['Structure'] == "endosomes")],
    dataset[(dataset['Structure'] == "ER (SERCA2)")],
    dataset[(dataset['Structure'] == "ER (Sec61 beta)")],
    dataset[(dataset['Structure'] == "Golgi")]
])
print(f'Number of cells: {len(soi_df)}')
print(f'Number of columns: {len(soi_df.columns)}')

Number of cells: 65108
Number of columns: 77


In [23]:
cell_stages = soi_df.cell_stage.unique()
sois = soi_df.Structure.unique()

samples = []

for stage in cell_stages:
    for structure in sois:
        samples.append(soi_df[
            (soi_df['cell_stage'] == stage) &
            (soi_df['Structure'] == structure)
        ].sample(5))
    
samples_for_visual_evaluation = pd.concat(samples)
print(f'Number of cells: {len(samples_for_visual_evaluation)}')
print(f'Number of columns: {len(samples_for_visual_evaluation.columns)}')

Number of cells: 225
Number of columns: 77


In [26]:
samples_for_visual_evaluation.sort_values(['Structure', 'cell_stage'])
samples_for_visual_evaluation.to_csv('cells_to_look_at.csv')

In [35]:
def generate_imaging_url(cell_id):
    return f"https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id={cell_id}"

In [36]:
for index, sample in samples_for_visual_evaluation.iterrows():
    print(index, sample['Structure'], sample['cell_stage'], generate_imaging_url(index))

262169 microtubules M0 https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id=262169
241493 microtubules M0 https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id=241493
879494 microtubules M0 https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id=879494
900596 microtubules M0 https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id=900596
249497 microtubules M0 https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id=249497
330541 mitochondria M0 https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id=330541
412091 mitochondria M0 https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id=412091
277349 mitochondria M0 https://allen-cell-animated.github.io/website-3d-cell-viewer/?dataset=aics_hipsc_v2021.1&id=277349
346218 mitochondria M0 h